## Advent of code 2019 day 1-10
See https://adventofcode.com/

In [ ]:
# note that this notebook requires the .venv-pypy environment for pypy 3.9
# to activate it from a git bash shell: source .venv-pypy/Scripts/activate
# to generate its requirements: pip freeze > .venv-pypy-requirements.txt

import collections
import itertools
import re
import copy
import math
import sys
import time
import json
import cProfile

In [ ]:
# utility functions and version check

def get_line_groups(lines):
    '''return list of lists of lines, each separated by empty lines, ignores empty lines from start and end'''
    lines=list(lines)
    lines.append('') # add terminator
    res=[]
    group=[]
    for line in lines:
        line=line.strip()
        if len(line)>0:
            group.append(line)
        elif len(group)>0: # close group
            res.append(group)
            group=[]
    return res

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

def exit():
    raise StopExecution()
    
print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}') # supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # 2019 day 2), then whenever you want save
# a private leaderboard json file, and run python privaterank.py filename.json

In [ ]:
# 2019 day 3
# start_ts=1661581356
# mv ~/Downloads/input* data_src/2019-day-3-input.txt
# big input file looks like: list of codes on two lines
# idea: part 1 parse and split each line, draw into grid, count lines per cell, then list crossoverpoints and take closest

sample1='''
R8,U5,L5,D3
U7,R6,D4,L4
'''

sample2='''
R75,D30,R83,U83,L12,D49,R71,U7,L72
U62,R66,U55,R34,D71,R55,D58,R83
'''

sample3='''
R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
U98,R91,D20,R16,D67,R40,U7,R15,U6,R7
'''

def update_cell(grid, x, y, lineno, steps):
    cell=grid.setdefault( (x, y), {})
    if lineno in cell:
        return 
    cell[lineno]=steps

def draw_lines(grid, lines, lineno):
    x=0
    y=0
    steps=0
    for line in lines:
        direction=line[0]
        distance=int(line[1:])
        for i in range(distance):
            if direction=='U':
                y-=1
            elif direction=='D':
                y+=1
            elif direction=='L':
                x-=1
            elif direction=='R':
                x+=1
            steps+=1
            update_cell(grid, x, y, lineno, steps)

def crossovers_dist(grid): # for part 1
    crossovers=[ k for k,v in grid.items() if len(v)>1 ]
    cr_dists=[ abs(tup[0])+abs(tup[1]) for tup in crossovers ]
    cr_dists.sort()
    return cr_dists

def steps_dist(grid): # for part 2
    crossovers=[ k for k,v in grid.items() if len(v)>1 ]
    cr_steps=[ grid[tup][0]+grid[tup][1] for tup in crossovers ]
    cr_steps.sort()
    return cr_steps

sample1=open('data_src/2019-day-3-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ lin.split(',') for lin in lines ]
assert len(data)==2
grid={} # maps x,y tuple to map of lineno to steps to reach
draw_lines(grid, data[0], 0)
draw_lines(grid, data[1], 1)
res=steps_dist(grid)
print(res[0])

# 27330

In [ ]:
# 2019 day 2
# start_ts=1643360000
# mv ~/Downloads/input data_src/2019-day-2-input.txt
# big input file looks like: about 100 numbers
# idea: part 1 parse as single list, then simply run

sample1='''
1,9,10,3,2,3,11,0,99,30,40,50
'''

sample2='''
1,1,1,4,99,5,6,0,99
'''

def run_opcodes(data):
    i=0
    while True:
        if data[i]==1:
            a=data[data[i+1]]
            b=data[data[i+2]]
            data[data[i+3]]=a+b
            i+=4
        elif data[i]==2:
            a=data[data[i+1]]
            b=data[data[i+2]]
            data[data[i+3]]=a*b
            i+=4
        elif data[i]==99:
            break
        else:
            assert False

sample1=open('data_src/2019-day-2-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data0=[ int(s) for s in lines[0].split(',') ]
for noun in range(100):
    for verb in range(100):
        data=list(data0)
        data[1]=noun
        data[2]=verb
        run_opcodes(data)
        if data[0]==19690720:
            answer=100*noun+verb
            print(f'{answer=}')
            exit()

In [ ]:
# 2019 day 1
# start_ts=1661070296
# mv ~/Downloads/input* data_src/2019-day-1-input.txt
# big input file looks like: list of numbers
# idea: part 1 parse ..., then ...

sample1='''
100756
'''

def fuel(mass):
    res=0
    while mass>0:
        fuel=mass//3 - 2
        if fuel>=0:
            res+=fuel
            mass=fuel
        else:
            break
    return res

sample1=open('data_src/2019-day-1-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ int(s) for s in lines ]
data=[ fuel(i) for i in data ]
sum(data) # should be 5003530

In [ ]:
# TEMPLATE
# 2019 day 6
# start_ts=RUN FIRST CELL TO GET TIME CODE BEFORE OPENING THE ASSIGNMENT
# mv ~/Downloads/input* data_src/2019-day-6-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample1='''

'''

#sample1=open('data_src/2019-day-6-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ int(s) for s in lines[0].split(',') ]
groups=get_line_groups(lines)
data0=[ s.split() for s in lines ]
data0=[ [cmd, int(num), 0] for cmd, num in data0 ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'(\w+)\s*x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)', s)) ]
data=[ (row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6]) ) for row in data ]
# template, remove what's not needed